## Линейные методы. Vowpal Wabbit.

Vowpal Wabbit on GitHub: https://github.com/JohnLangford/vowpal_wabbit

Vowpal Wabbit Tutorial: https://github.com/JohnLangford/vowpal_wabbit/wiki/Tutorial

In [10]:
import pandas as pd
import numpy as np

In [11]:
train_path = 'train-sample.csv'

In [12]:
data = pd.read_csv(train_path)
data.head()
print(len(data))

140272


In [13]:
print(data.OpenStatus[10])
print(data.BodyMarkdown[10])

open
i have two dimensions, first (width, height) and second(width1, height1). how can i retrieve a Point(x,y) from dimensions???


In [14]:
data_train = data.iloc[:50000, :]
data_test = data.iloc[70000:, :]

In [6]:
import re

def save_to_vw(data, fname):
    with open(fname, 'w') as fout:
        for _, row in data.iterrows():
            text = filter(lambda x: len(x) > 1, re.split("[^a-z]",
                                    row.BodyMarkdown.lower()))
            text = ' '.join(text)
            if row.OpenStatus == "open":
                target = 1
            else:
                target = -1
            fout.write('{0} |n 0:{1} {2} |t {3}\n'.format(target, 
                                        row.ReputationAtPostCreation,
                                        row.Tag1,
                                        text))

In [7]:
save_to_vw(data_train, 'train.vw')
save_to_vw(data_test, 'test.vw')

In [8]:
!head -n 2 train.vw

1 |n 0:1 mongodb |t am building corpus of indexed sentences in different languages have collection of languages which have both an objectid and the iso code as key is it better to use reference to the language collection or store key like en or fr suppose it compromise between ease of referencing the language object in that collection speed in doing queries where the sentence has certain language the size of the data on disk any best practices that should know of
1 |n 0:192 dom |t create xml document with jaxp and search way to insert the schemalocation at the moment my application produces xml version encoding utf root root but need xml version encoding utf root xmlns namespaceurl xmlns xs http www org xmlschema instance xs schemalocation namespaceurl pathtomyschema xsd root my code streamresult result new streamresult writer document doc getdocument transformer trans transfac newtransformer trans setoutputproperty outputkeys indent yes trans setoutputproperty outputkeys method xml tr

In [9]:
!vw -d train.vw -c -k -f model.vw --passes 10 

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000       81
0.500000 0.000000            2            2.0   1.0000   1.0000       98
0.906326 1.312652            4            4.0  -1.0000   0.4266      110
0.821724 0.737121            8            8.0  -1.0000   0.4654       38
0.875065 0.928406           16           16.0   1.0000   0.7270      136
0.906826 0.938586           32           32.0   1.0000   0.5103       93
0.996030 1.085234           64           64.0   1.0000   0.7147       71
0.977266 0.958502          128          128.0  -1.0000  -0.3218       24
0.990424 1.003582          256          256.0  -1.0000   0.2728      

In [11]:
!vw -d test.vw -i model.vw -t -p pred.txt

Generating 2-grams for t namespaces.
only testing
predictions = pred.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0   1.0000   1.0000      370
0.000000 0.000000            2            2.0   1.0000   1.0000      248
1.010778 2.021556            4            4.0  -1.0000   1.0000     1028
0.660964 0.311150            8            8.0   1.0000   1.0000      230
0.535013 0.409062           16           16.0   1.0000   0.5860      306
0.523252 0.511491           32           32.0   1.0000  -0.0752       92
0.690913 0.858574           64           64.0  -1.0000   0.3738      448
0.662274 0.633634          128          128.0   1.0000   0.2049      168
0.702460 0.742647          256          256.0  -1.0000   0.0076      1

In [12]:
!head -n 10 pred.txt

1
1
-1
1
0.082040
1
-1
0.805499
-0.150867
-0.643998


In [17]:
from sklearn.metrics import roc_auc_score

def calc_vw_qual():
    preds = pd.read_csv('pred_tutorial.txt', header=None).iloc[:, 0].values
    target = data_test.OpenStatus.values
    T = []
    for t in target:
        if t == 'open':
            T.append(1.)
        else:
            T.append(-1.)
    print(roc_auc_score(T, preds))
    
calc_vw_qual()

0.784259407516


In [25]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1
!vw -d test.vw -i model.vw -t -p pred.txt
print('\n\n\n')
calc_vw_qual()

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000       81
0.500000 0.000000            2            2.0   1.0000   1.0000       98
0.812846 1.125692            4            4.0  -1.0000   0.2230      110
0.769143 0.725440            8            8.0  -1.0000   0.2957       38
0.845550 0.921957           16           16.0   1.0000   0.4906      136
0.812974 0.780397           32           32.0   1.0000   0.5922       93
0.899824 0.986675           64           64.0   1.0000   0.3493       71
0.894960 0.890097          128          128.0  -1.0000  -0.0987       24
0.893272 0.891584          256          256.0  -1.0000   0.0260      

n-граммы (n=2) - индикаторы того, что два слова встретились рядом. Из "мама мыла раму" получаем биграммы "мама мыла" и "мыла раму".

In [26]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2
!vw -d test.vw -i model.vw -t -p pred.txt
print('\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000      158
0.501991 0.003982            2            2.0   1.0000   0.9369      192
0.769028 1.036065            4            4.0  -1.0000   0.1032      216
0.788365 0.807701            8            8.0  -1.0000   0.2224       72
0.853561 0.918757           16           16.0   1.0000   0.3748      268
0.815435 0.777309           32           32.0   1.0000   0.5281      182
0.899736 0.984037           64           64.0   1.0000   0.1909      138
0.899076 0.898416          128          128.0  -1.0000  -0.0599       44
0.900915 0.902754          256  

k-skip-n-граммы - как n-граммы, но разрешаем словам быть отдаленными друг от друга не больше, чем на k

In [27]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 --skips t2
!vw -d test.vw -i model.vw -t -p pred.txt
print('\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
Generating 2-skips for t namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000      309
0.598903 0.197807            2            2.0   1.0000   0.5552      377
0.813193 1.027482            4            4.0  -1.0000   0.0751      425
0.827788 0.842383            8            8.0  -1.0000   0.1551      137
0.878166 0.928544           16           16.0   1.0000   0.2804      529
0.833454 0.788743           32           32.0   1.0000   0.4647      357
0.919097 1.004739           64           64.0   1.0000   0.1906      269
0.925038 0.930980          128          128.0  -1.0000  -0.0388     

In [28]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 -b 28
!vw -d test.vw -i model.vw -t -p pred.txt
print('\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000      158
0.501991 0.003982            2            2.0   1.0000   0.9369      192
0.768823 1.035654            4            4.0  -1.0000   0.1028      216
0.788250 0.807676            8            8.0  -1.0000   0.2224       72
0.852059 0.915868           16           16.0   1.0000   0.3741      268
0.814879 0.777699           32           32.0   1.0000   0.5283      182
0.897638 0.980397           64           64.0   1.0000   0.2017      138
0.897445 0.897252          128          128.0  -1.0000  -0.0599       44
0.898531 0.899617          256  